In [1]:
#Import Required Libraries:
import pandas as pd
import networkx as nx
import json

#Load Author Network Data
df = pd.read_csv("E:\data_scopus.csv")

nodes = df['EID'].values
df=df.fillna(0)
df.head(2)

,Title,Year,EID,Abstract,Publisher,Conference name,Conference date,Authors,Author(s) ID,Authors with affiliations,Source title,Abbreviated Source Title,Cited by
0,Virtual reality applications for the built env...,2020,2-s2.0-85086464158,With its advanced capabilities of immersive an...,Elsevier B.V.,0,0,"Zhang Y., Liu H., Kang S.-C., Al-Hussein M.",57202536919;53984550800;57215426514;6603541102;,"Zhang, Y., Department of Civil and Environment...",Automation in Construction,Autom Constr,0.0
1,"Self-tracking while doing sport: Comfort, moti...",2020,2-s2.0-85082875828,The spread of wearable technologies is paving ...,Academic Press,0,0,"Rapp A., Tirabeni L.",23398572100;57191836654;,"Rapp, A., Computer Science Department, Univers...",International Journal of Human Computer Studies,Int J Hum Comput Stud,0.0


In [5]:
df = df.dropna(subset=['Author(s) ID', 'Year', 'Authors', 'Authors with affiliations'])
df = df.fillna(0)

# Initialize the graph
G = nx.Graph()

# Iterate over each row to add nodes and their attributes
for _, row in df.iterrows():
    authors = row['Authors'].split(',')
    author_ids = row['Author(s) ID'].split(';')
    title = row['Title']
    year = row['Year']
    citations = row['Cited by']
    publisher = row['Publisher']
    author_with_affiliations = row['Authors with affiliations']

    # Add each author as a node
    for index, author_id in enumerate(author_ids):
        if author_id.strip():
            G.add_node(
                author_id,
                id=author_id,
                Authors=', '.join(authors),
                Title=title,
                Year=year,
                Citations=citations,
                Publisher=publisher,
                Author_with_affiliations=author_with_affiliations
            )

# Output the graph as a JSON file
with open("publication_network.json", "w") as f:
    json.dump(json_graph.node_link_data(G), f)

print("Graph and JSON file creation completed successfully!")

Graph and JSON file creation completed successfully!


In [12]:
# Load the data from the JSON file
with open(r'C:\Users\Hitesh\Downloads\publication_network.json', 'r') as infile:
    data = json.load(infile)

# Initialize a new NetworkX graph
G = nx.Graph()

# Function to extract the country from the first affiliation
def get_author_country(authors_with_affiliations):
    first_affiliation = authors_with_affiliations.split(';')[0].strip()
    country = first_affiliation.split(',')[-1].strip()
    return country

# Add nodes (publications) to the graph with attributes
for node_data in data['nodes']:
    eid = node_data.get('id')
    authors = node_data.get('Authors')
    title = node_data.get('Title')
    year = node_data.get('Year')
    citations = node_data.get('Citations')
    publisher = node_data.get('Publisher')
    authors_with_affiliations = str(node_data.get('Author with affiliations'))

    # Extract the country using the helper function
    country = get_author_country(authors_with_affiliations)

    # Add the node to the graph
    G.add_node(
        eid,
        authors=authors,
        title=title,
        year=year,
        citations=citations,
        publisher=publisher,
        authors_with_affiliations=authors_with_affiliations,
        country=country
    )

# Generate clusters based on authors' countries
clusters = list(nx.connected_components(G))

# Assign classes to each cluster
class_mapping = {node: idx for idx, cluster in enumerate(clusters) for node in cluster}
nx.set_node_attributes(G, class_mapping, 'class')

# Create a dictionary to store co-authorship relationships
for _, row in df.iterrows():
    authors = row['Author(s) ID'].split(';')
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            if authors[i].strip() and authors[j].strip():  # Ensure no empty strings
                G.add_edge(authors[i], authors[j])

# Save the graph data as a JSON file with proper character encoding
coauthorship_data = {
    'nodes': [{'id': node, **G.nodes[node]} for node in G.nodes()],
    'links': [{'source': source, 'target': target} for source, target in G.edges()]
}

with open('coauthorship_data.json', 'w', encoding='utf-8') as outfile:
    json.dump(coauthorship_data, outfile, ensure_ascii=False)

print("Graph and JSON file creation completed successfully!")

Graph and JSON file creation completed successfully!
